This is a notebook to be used for the Capstone project in IBM Data Science Specialization on Coursera

In [4]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [5]:
import pandas as pd
import numpy as np

In [6]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [7]:
from bs4 import BeautifulSoup
import requests

In [8]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

r = requests.get(url).text

In [9]:
soup = BeautifulSoup(r, 'html.parser')

In [10]:
# Save table from soup in a variable
data = soup.table

# Create empty list
listOfEvt = []

# Loop all 'tr' elements and add all 'td elements to list'
for tr in data.find_all('tr'):
    for td in tr.find_all('td'):
        listOfEvt.append(td.text.rstrip())

In [51]:
# Table is postcode, borough, neighbourhood but assignment is as follows
column_names = ['Postalcode', 'Neighbourhood', 'Borough'] 

# Instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [52]:
# Run through the list of everything, incrementing by 3 each loop.
for i in range(0,len(listOfEvt),3):    
    
    # Set variables
    postcode = listOfEvt[i]
    neighborhood_name = listOfEvt[i+2]
    borough = listOfEvt[i+1]
    
    # Make sure borough is not 'not assigned'
    if(borough == 'Not assigned'):
        a = 1
    else:
        # Make sure neighborhood is not 'not assigned'
        if(neighborhood_name == 'Not assigned'):
            neighborhood_name = borough
        
        # add to dataframe
        neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighbourhood': neighborhood_name,
                                          'Postalcode': postcode}, ignore_index=True)

In [54]:
# Sort by postal code
neighborhoods.sort_values(['Postalcode'], inplace=True)
neighborhoods = neighborhoods.reset_index()
neighborhoods.drop(['index'], axis=1, inplace=True)
neighborhoods.head()

,Postalcode,Neighbourhood,Borough
0,M1B,Rouge,Scarborough
1,M1B,Malvern,Scarborough
2,M1C,Port Union,Scarborough
3,M1C,Rouge Hill,Scarborough
4,M1C,Highland Creek,Scarborough


In [65]:
# Categorize dataframe by postalcode
df = neighborhoods.groupby(['Postalcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df.head()

,Postalcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [66]:
# Shape of Dataframe
df.shape

(103, 3)